**note**: temp notebook, will merge into the main t5.ipynb notebook later

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# %pip install transformers
# %pip install sentencepiece
# !cp /content/drive/MyDrive/TED_Project/Data_output/t5.py t5.py
# import t5
# from t5 import *


##########t5.py##########


#import libraries
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from tqdm import tqdm
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
#set up tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
    
def text_summarizer(transcript):
    """
    input: text
    output: 10-80-word summary of text
    """
    transcript = transcript.strip().replace("\n"," ")
    inputs = tokenizer.encode("summarize: " + transcript,
                              return_tensors='pt',
                              max_length=len(transcript.split()),
                              truncation=True)
    summarization_ids = model.generate(inputs, max_length=80, min_length=10,
                                       no_repeat_ngram_size=3,
                                       length_penalty=5., num_beams=1,
                                       early_stopping=True)
    summarization = tokenizer.decode(
        summarization_ids[0], skip_special_tokens=True)
    return summarization
    
def similarity(stsb_sentence_1, stsb_sentence_2):
    """
    input: 2 texts
    output: similarity score 0-5, with 5 as maximum similarity and 0 as no similarity
    """
    input_ids = tokenizer("stsb sentence 1: "+stsb_sentence_1+" sentence 2: "+stsb_sentence_2, return_tensors="pt").input_ids
    stsb_ids = model.generate(input_ids)
    stsb = tokenizer.decode(stsb_ids[0],skip_special_tokens=True)
    return stsb


def create_similarity_matrix(df, feature):
    """
    input: dataframe with 'summary' feature
    returns similarity matrix by cross-comparing the rows and column values
    """
    # values = df['summary'].values # values will be the sentence summaries
    values = df[feature].values
    # make a df with rows and column names from summary columns
    df_summary = pd.DataFrame(index=values, columns=values)
    
    # loop through rows & columns to do similarity (row,col)
    df = df_summary.copy()
    for row_idx, row in tqdm(enumerate(df.index)):
        for col in df.columns[row_idx:]:
            # similarity function
            df.loc[row,col]=similarity(row, col) 
    # mirror the df across the diagonal        
    for row_idx, row in enumerate(df.index):
        for col in df.columns[row_idx:]:
            df.loc[col,row] = df.loc[row,col]
    return df

# sim_matrix = similarity_matrix(df_summary)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [3]:
# df = pd.read_csv('/content/drive/MyDrive/TED_Project/Data_output/ted_talk_clean_merged_bert.csv',
#                  parse_dates=['date']).drop(columns=['Unnamed: 0'])


df = pd.read_csv('../../../Data_output/ted_talk_clean_merged_bert.csv',
                 parse_dates=['date']).drop(columns=['Unnamed: 0'])
df.head(2)

/var/folders/yj/c676wyw500q1vyq2brsvcqtr0000gn/T/ipykernel_82716/2394093975.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('../../../Data_output/ted_talk_clean_merged_bert.csv',


,author,title,description,likes,views,transcript,date,tags
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...",2023-06-01,"art, creativity, design, fashion, performance"
1,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,2023-06-01,"diversity, identity, inclusion, indigenous_peo..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5140 entries, 0 to 5139
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   author       5140 non-null   object        
 1   title        5140 non-null   object        
 2   description  5140 non-null   object        
 3   likes        5140 non-null   int64         
 4   views        5140 non-null   int64         
 5   transcript   5140 non-null   object        
 6   date         5140 non-null   datetime64[ns]
 7   tags         5140 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 321.4+ KB


In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['summary']=None
for idx in tqdm(df.index):
    df.loc[idx,'summary'] = text_summarizer(df.loc[idx,'transcript'])

In [ ]:
df[df['summary'].isna()]

In [ ]:
df['summary'][df['summary'].isna()] = "This is a musical performance, with no spoken words."
df['summary'].isna().sum()

In [ ]:
df.to_csv('/content/drive/MyDrive/TED_Project/Data_output/ted_talk_summary_t5_v2.csv')

In [3]:
df = pd.read_csv('../../../Data_output/ted_talk_summary_t5_v2.csv',
                 parse_dates=['date']).drop(columns=['Unnamed: 0'])
df.head()

,author,title,description,likes,views,transcript,date,tags,summary
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...",2023-06-01,"art, creativity, design, fashion, performance",a 24-hour show is a '24 Decades: a history of ...
1,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,2023-06-01,"diversity, identity, inclusion, indigenous_peo...",a queer culture is a part of dominant queer cu...
2,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...,2023-06-01,"business, community, equality, humanity, money...",philanthropy is a radically different type of ...
3,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R...",2023-06-01,"africa, animation, education, history, identit...",a tenth of the country's population was killed...
4,Conor Russomanno,a powerful new neurotech tool for augmenting y...,in an astonishing talk and tech demo neurotech...,1100,374259,I became obsessed with the relationship betwee...,2023-06-01,"biotech, brain, disability, health, invention,...",a new technology is being developed to measure...


In [5]:
# df['soup'] = df['description'] + '. ' + ['summary'] + '. ' + df['tags'] +'.'
df['soup'] = df['description'] +  '. ' + df['tags'] +'.'
df.head(3)

,author,title,description,likes,views,transcript,date,tags,summary,soup
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...",2023-06-01,"art, creativity, design, fashion, performance",a 24-hour show is a '24 Decades: a history of ...,tapping into the transformational power of cos...
1,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,2023-06-01,"diversity, identity, inclusion, indigenous_peo...",a queer culture is a part of dominant queer cu...,how can we move past societys inclination to b...
2,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...,2023-06-01,"business, community, equality, humanity, money...",philanthropy is a radically different type of ...,is there a way to give back that benefits ever...


In [ ]:
sim_matrix = create_similarity_matrix(df, 'soup')

0it [00:00, ?it/s]/Users/uche/miniconda3/envs/lhl_env/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/Users/uche/miniconda3/envs/lhl_env/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
67it [55:52:43, 3002.44s/it]


In [ ]:
sim_matrix[:10]

In [ ]:
def recommender(row, number_of_recs, sim_matrix=sim_matrix, df=df):
    """
    input: the index of the title from the ted_talk_csv
    output: a list of recommended titles and their tags
    """
    list_of_ = np.argsort(sim_matrix.iloc[row].values)[::-1]
    return df.iloc[list_of_][1:(number_of_recs+1)][['title','tags']]

In [ ]:
recommender(2)